In [1]:
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import requests
import csv
import itertools
import re
import numpy as np
import urllib3
import math
import datetime

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

from urllib.parse import urlparse, urlunparse

In [2]:
# 關閉 urllib3 函式庫在執行 HTTPS 請求時可能會發出的警告
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 尋找一個 .env 文件，並且讀取文件中的所有變量，然後把它們加到你的環境變量中
load_dotenv()

False

In [3]:
#### 原始的爬取方式(沒有json檔之前)
# def get_search_page_count(url):

#     res = requests.get(url)

#     soupSearch = BeautifulSoup(res.text, 'lxml')  # 以上是網路獲取html

#     # 搜尋頁面總頁數
#     searchPages = soupSearch.find_all("a", class_="rlp-main-pagination__btn-txt")
    
#     # 搜尋每頁總比數
#     searchPageCount = soupSearch.find_all("div", class_="rlp-main-hotel__container")

#     # 取得搜尋頁面總頁數 跑迴圈用
#     searchLastPageCount = int(searchPages[len(searchPages)-1].string)
    
#     # 取得每個頁面總筆數
#     searchCountPerPage = len(searchPageCount)

#     return searchLastPageCount,searchCountPerPage

# # 跑迴圈將搜尋飯店評論的 url 整理成一維 list
# def loop_search_hotel_comments(url, pageCount, CountPerPage):

#     hotelNames = []
#     hotelCommentUrls = []
#     plusOffset = 0
#     searchUrl = ""

#     for i in range(int(pageCount)):
#         if(i == 0):
#             searchUrl = url
#         else:
#             plusOffset = plusOffset + CountPerPage
#             searchUrl = url+str(";offset=")+str(plusOffset)+str("&")

#         loopPageSearchGet = requests.get(searchUrl)

#         searchResult = BeautifulSoup(loopPageSearchGet.text, 'lxml')  # 以上是網路獲取html

#         # 搜尋頁面飯店名稱
#         hotelNames.append(searchResult.find_all("a", class_="rlp-main-hotel__hotel-name-link"))

#         # 搜尋頁面飯店評論 url
#         hotelCommentUrls.append(searchResult.find_all("li", class_="rlp-main-hotel-review__review_link"))

           
#     # 利用套件 itertools 合併一維 list
#     return [
#         list(itertools.chain.from_iterable(hotelCommentUrls)),
#         list(itertools.chain.from_iterable(hotelNames)),
#     ]

#### 使用 json 檔的 list 爬取 
# 讀取 json 檔案
def readJson():
    
    # 開啟 json 檔案
    with open('./booking_hotels_list.json', "r", encoding='utf-8') as file:
        
        # 取得 json 字串
        strJson = file.read()
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    return listResult

# 正式爬蟲顧客對飯店的評論資訊(start 與 end 表示本次想要爬取的飯店起始間數與結束間數)
def loop_formal_reptile_hotel＿comments(listResult,start,end):
    
    # 放所有爬取的評論
    commentList = {}

    key = 0

    for obj in listResult[(start-1):end]:

        key = key + 1
        
        # 初始化 list of index 'key'
        commentList[key] = []
        
        url = obj['url']
        # print("原始網址(飯店網址):",url)
        
        # 解析網址(拆解網址)
        parsed_url = urlparse(url)

        # 將解析出的路徑（path）部分分割成各部分，插入'reviews'並把 'hotel' 的位置調整
        path_parts = parsed_url.path.split('/')
        path_parts[1] = 'reviews'
        path_parts.insert(3, 'hotel')

        # 重新組合路徑
        new_path = "/".join(path_parts)

        # 建立新的網址(組合網址)
        new_url = urlunparse((parsed_url.scheme, parsed_url.netloc, new_path, parsed_url.params, parsed_url.query, parsed_url.fragment))
        
        # 建立評論頁面網址
        # 範例 : https://www.booking.com/reviews/tw/hotel/w3-hostel.zh-tw.html?r_lang = all
        hotelUrl = str(new_url) + str('?r_lang=all;rows=25&')  # r_lang = all (全部語言) 或 r_lang = zh-tw (中文)
        # print("評論網址:",hotelUrl)
        
        # 接 requests exception
        try:
            
            # 自訂標頭
            headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
                      'Accept-Language': 'zh-TW,zh;q=0.9'}
            
            # 獲取 html
            hotel = requests.get(hotelUrl, headers=headers, timeout=5)
            
            # bs 解析(當伺服器端回傳的內容為 html 檔案而不是 json 檔時使用)
            searchHotelResult = BeautifulSoup(hotel.text, 'lxml')

            # 取得飯店總評論比數
            commentTotalCountString = searchHotelResult.find("p", class_="review_list_score_count")
            
            # 如果飯店評論比數是 0 就跳過，執行下一間
            if commentTotalCountString is None:
                continue
            
            commentTotalCountString = commentTotalCountString.string
            
            # 將所有非數字刪除
            commentTotalCount = re.sub("\D", "", commentTotalCountString)
            
            # 放一間飯店的評論
            o = []
            
            # 跑分頁迴圈
            for page in range(math.ceil(int(commentTotalCount)/25) + 1):
                
                if(int(page) == 0):
                    sendUrl = hotelUrl
                else:
                    if(int(page) != 1):
                        sendUrl = hotelUrl+str(";page=")+str(int(page))
                    else:
                        continue
                        
                print(sendUrl)
                
                loopHotelPageRes = requests.get(str(sendUrl), headers=headers, verify=False)
                sleep(0.5)
                loopHotelPageResult = BeautifulSoup(loopHotelPageRes.text, 'lxml')
                
                # 當頁數是第一頁時才需要做
                if(int(page) == 0):
                    
                    # 飯店名稱(只有一個)
                    htName = loopHotelPageResult.select_one("a.standalone_header_hotel_link").get_text()
                    
                    # 整體評分(只有一個)
                    totalScore = loopHotelPageResult.select_one("span.review-score-badge").get_text()
                    
                    # 單項名稱(有 7 個或 6 個)
                    # singleName = [sn.get_text() for sn in loopHotelPageResult.find_all("p", class_="review_score_name")]
                    
                    # 單項評分(有 7 個或 6 個)
                    singleScore = [ss.get_text() for ss in loopHotelPageResult.find_all("p", class_="review_score_value")]
                    
                    # 地址(一個)
                    address = [ad.get_text() for ad in loopHotelPageResult.find_all("p", class_="hotel_address")]
                    
                    # booking 參考排名(縣市)(地區) (一個)
                    rankTitle = ''
                    if(len(loopHotelPageResult.find_all("a", class_="hotel_rank_link")) == 0):
                        rankTitle = ''
                    else:
                        rankTitle = loopHotelPageResult.select_one("a.hotel_rank_link").get_text()
                    
                    # booking 參考排名(名次) (一個)
                    rank = ''
                    if(len(loopHotelPageResult.find_all("strong", class_="number_highlight")) == 0):
                        rank = ''
                    else:
                        rank = loopHotelPageResult.select_one("strong.number_highlight").get_text()
                    
                    # 排名總間數 (一個)
                    totalHotel = ''
                    if(len(loopHotelPageResult.find_all("p", class_="hotel_rank")) == 0):
                        totalHotel = ''
                    else:
                        totalHotel = loopHotelPageResult.select_one("p.hotel_rank").get_text()
                    
                    # 總評論數 (一個)
                    totalCount = [re.sub("\D", "", ttc.get_text()) for ttc in loopHotelPageResult.find_all("p", class_="review_list_score_count")]


                # 留言姓名
                commentNames = [name.get_text().replace('\n', "").replace('\r', "") for name in loopHotelPageResult.find_all("p", class_="reviewer_name")]

                # 國籍
                commentCountries = [country.get_text().strip('\n') for country in loopHotelPageResult.find_all("span", class_="reviewer_country")]

                # 留言推薦
                commentUserReviews = [re.sub("\D", "", review.get_text().strip('\n')) for review in loopHotelPageResult.find_all("div", class_="review_item_user_review_count")]

                # 評分
                commentReviewScoreBadges = [score.get_text().replace('\n', "").replace('\r', "") for score in loopHotelPageResult.find_all("div", class_="review_item_header_score_container")]

                # 留言標題
                commentContentHeaders = [header.get_text().replace('\n', "").replace('\r', "") for header in loopHotelPageResult.find_all("div", class_="review_item_header_content_container")]

                # 填寫日期
                insertDates = [re.sub("\D", "", date.get_text()) for date in loopHotelPageResult.find_all("p", class_="review_item_date")]

                # 標籤
                infoTags = loopHotelPageResult.find_all("ul", class_="review_item_info_tags")

                # 評語
                commentContents = loopHotelPageResult.find_all("div", class_="review_item_review_content")
                
                
                # 到後面沒資料停止，換跑下一間飯店
                if(len(commentNames) == 0):
                    break
                else:
                    # 對一頁中的所有評論做處理
                    index = 0
                    for commentContent in commentContents:
                        
                        z = len(o)  # 尋找相同陣列 key
                        
                        # 第一次的陣列 append
                        o.append([commentNames[index]])  # 姓名
                        o[z].append(commentCountries[index])  # 國籍
                        o[z].append(commentUserReviews[index])  # 推薦數
                        o[z].append(commentReviewScoreBadges[index])  # 評分
                        o[z].append(commentContentHeaders[index])  # 留言標題
                        o[z].append(insertDates[index])  # 填寫日期

                        # 標籤住房
                        tags = infoTags[index].find_all("li", class_='review_info_tag')
                        tagValues = [tag.get_text().replace('•', "").replace('\n', "").replace('\r', "") for tag in tags]
                        tagValue = ",".join(tagValues)
                        o[z].append(tagValue)

                        # 壞留言
                        badValue = ""
                        if(commentContent.find("p", class_='review_neg') == None):
                            badValue = ""
                        else:
                            badValue = commentContent.find("p", class_='review_neg').get_text().replace('\n', "").replace('\r', "")
                        o[z].append(badValue)

                        # 好留言
                        goodValue = ""
                        if(commentContent.find("p", class_='review_pos') == None):
                            goodValue = ""
                        else:
                            goodValue = commentContent.find("p", class_='review_pos').get_text().replace('\n', "").replace('\r', "")
                        o[z].append(goodValue)
                        
                        # 入住時間
                        checkInTime = ""
                        if(commentContent.find("p", class_='review_staydate') == None):
                            checkInTime = ""
                        else:
                            checkInTime = re.sub("\D", "", commentContent.find("p", class_='review_staydate').get_text())
                        o[z].append(checkInTime)
                        
                        # 整體評分
                        o[z].append(totalScore)
                        
                        # 地址
                        o[z].append(address[0])
                        
                        # booking 參考排名(縣市)(地區)
                        o[z].append(rankTitle)
                        
                        # booking 參考排名(名次)
                        o[z].append(rank)
                        
                        # 排名總間數 
                        o[z].append(totalHotel)
                        
                        # 總評論數
                        o[z].append(totalCount[0])
                        
                        # 單項評分
                        for x in singleScore:
                            o[z].append(x)
                        
                        # 單項評分有缺就補空值
                        if len(singleScore) < 7:
                            for j in range(7-len(singleScore)):
                                o[z].append('')
                            
                        # 飯店名稱
                        o[z].append(htName)
                        
                        # 加入 json 檔資訊
                        for y in obj:
                            o[z].append(obj[y])

                        index = index + 1

            commentList[key].append(o)

        except requests.exceptions.RequestException as e:

            print(e)

    return [commentList, [name['name'] for name in listResult[(start-1):end]]]

In [7]:
if __name__ == "__main__":
    
    listResult = readJson()
    
    # 建立資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
    path = os.path.join(os.getcwd(), 'bookingHotelComments_3001_3500') # 目錄名稱改成自己的間數 (下一個就改 bookingHotelComments_501_1000)
    if not os.path.exists(path):
        os.makedirs(path)
        
    path = path + '/'
    
    # 統一符號
    path = path.replace('\\', '/') 

In [12]:
if __name__ == "__main__":
    
    # 開始客製化爬取評論並存成 csv
    
    # totalHotel = len(listResult)
    
    # 開始測量(爬取時間)
    startime = datetime.datetime.now()
    
    # 要開始爬取的飯店為第幾間
    start = 3201
    
    # 要結束的飯店為第幾間
    end = 3500
    
    # 下一個就設 start = 501  end = 1000
    # 再下一個就 start = 1001  end = 1500
    # 類推
    
    result = loop_formal_reptile_hotel＿comments(listResult,start,end)

    key = 0

    # 開始匯出 csv
    for i in result[0]:
        for data in result[0][i]:
            
            # 飯店名稱有 '/' 要改掉
            if '/' in str(result[1][key]):
                fn = str(result[1][key]).replace('/','_')
            else:
                fn = str(result[1][key])
            
            fileName = path + fn + '.csv'

            if '\t' in fileName:
                fileName = fileName.replace('\t', '')

            if '|' in fileName:
                fileName = fileName.replace('|', '_')
            
            key = key + 1
            with open(fileName, 'w', encoding="utf_8_sig", newline="") as csvFile:
                writer = csv.writer(csvFile)
                list = ["姓名", "國籍", "留言數", "評分", "留言標題", "填寫日期","標籤", "負評", "正評", "入住時間","整體評分","地址","縣市地區","參考排名","排名總間數","總評論數","單項評分_整潔度","單項評分_舒適程度","單項評分_住宿地點","單項評分_設施","單項評分_員工素質","單項評分_性價比","單項評分_免費WiFi","飯店名稱"]
                
                # 併入 json 檔的欄位名稱(隨便選一個即可)
                for obj in listResult[10:11]:
                    for y in obj:
                        list.append(y)
                        
                writer.writerow(list)
                for lists in data:
                    writer.writerow(lists)
    
    # 結束測量
    endtime = datetime.datetime.now()
    
    # 輸出結果
    print("執行時間：", endtime - startime)

https://www.booking.com/reviews/tw/hotel/song-ting.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/song-ting.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/song-ting.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/song-ting.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&;page=5
https://www.booking.com/reviews/tw/hotel/jia-ri-qi-che-lu-guan-jing-pin-guan.html?r_lang=all;rows=25&;page=6
https://www.booking.com/revi

https://www.booking.com/reviews/tw/hotel/chen-jian-wei.html?r_lang=all;rows=25&;page=6
https://www.booking.com/reviews/tw/hotel/buluba-dao-jia.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/buluba-dao-jia.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/buluba-dao-jia.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/buluba-dao-jia.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&;page=5
https://www.booking.com/reviews/tw/hotel/jia-yuan-qi-che-lu-guan.html?r_lang=all;rows=25&;

https://www.booking.com/reviews/tw/hotel/jing-shang-tien-hua.html?r_lang=all;rows=25&;page=10
https://www.booking.com/reviews/tw/hotel/jing-shang-tien-hua.html?r_lang=all;rows=25&;page=11
https://www.booking.com/reviews/tw/hotel/jing-shang-tien-hua.html?r_lang=all;rows=25&;page=12
https://www.booking.com/reviews/tw/hotel/zhu-yuan-shan-zhuang.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/hua-lian-shi-fu-bei-man-yi-min-su.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/hua-lian-shi-fu-bei-man-yi-min-su.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/hua-lian-shi-fu-bei-man-yi-min-su.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/hua-lian-shi-fu-bei-man-yi-min-su.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/hua-lian-shi-fu-bei-man-yi-min-su.html?r_lang=all;rows=25&;page=5
https://www.booking.com/reviews/tw/hotel/grand-day-resort.html?r_lang=all;rows=25&
https://www.booking.

https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=18
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=19
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=20
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=21
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=22
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=23
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=24
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=25
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=26
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=27
https://www.booking.com/reviews/tw/hotel/the-vita-house.html?r_lang=all;rows=25&;page=28
https://www.booking.c

https://www.booking.com/reviews/tw/hotel/tu-wei-192min-su.html?r_lang=all;rows=25&;page=9
https://www.booking.com/reviews/tw/hotel/tu-wei-192min-su.html?r_lang=all;rows=25&;page=10
https://www.booking.com/reviews/tw/hotel/tu-wei-192min-su.html?r_lang=all;rows=25&;page=11
https://www.booking.com/reviews/tw/hotel/tu-wei-192min-su.html?r_lang=all;rows=25&;page=12
https://www.booking.com/reviews/tw/hotel/zai-shui-fang.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/zai-shui-fang.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/new-km.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/new-km.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/zhong-zi-min-su-yuanshan.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/zhong-zi-min-su-yuanshan.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/guo-yuan-shang-wu-fan-dian.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/

...

https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=66
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=67
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=68
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=69
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=70
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=71
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=72
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=73
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=74
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=75
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;page=76
https://www.booking.com/reviews/tw/hotel/park-lees.html?r_lang=all;rows=25&;

https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=21
https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=22
https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=23
https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=24
https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=25
https://www.booking.com/reviews/tw/hotel/ai-ni-ya.html?r_lang=all;rows=25&;page=26
https://www.booking.com/reviews/tw/hotel/qing-ding-zhen-zhu.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/qing-ding-zhen-zhu.html?r_lang=all;rows=25&;page=2
https://www.booking.com/reviews/tw/hotel/qing-ding-zhen-zhu.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/da-fu-di-min-su.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/la-fleur-bnb-restaurant.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/la-fleur-bnb-restaur

https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=41
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=42
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=43
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=44
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=45
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=46
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=47
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=48
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=49
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=a

https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=122
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=123
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=124
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=125
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=126
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=127
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=128
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=129
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html?r_lang=all;rows=25&;page=130
https://www.booking.com/reviews/tw/hotel/luo-qi-nan-gang-da-fan-dian.html

https://www.booking.com/reviews/tw/hotel/xian-ren-ju-min-su.html?r_lang=all;rows=25&;page=3
https://www.booking.com/reviews/tw/hotel/xian-ren-ju-min-su.html?r_lang=all;rows=25&;page=4
https://www.booking.com/reviews/tw/hotel/xian-ren-ju-min-su.html?r_lang=all;rows=25&;page=5
https://www.booking.com/reviews/tw/hotel/xi-gang-wai-po-jia-min-su.html?r_lang=all;rows=25&
https://www.booking.com/reviews/tw/hotel/xi-gang-wai-po-jia-min-su.html?r_lang=all;rows=25&;page=2
執行時間： 2:53:21.722094


#### 檢查爬取狀況

In [28]:
# indices = [i for i, item in enumerate(listResult) if item['hotel_id'] == 3510215]
# print(indices)

[5174]


In [21]:
hotel = [item for item in listResult if item['name'] == 'J-HOTEL']
len(hotel)

1

In [4]:
if __name__ == "__main__":
    
    listResult = readJson()

In [5]:
for index,obj in enumerate(listResult):
    if 'Bitan Hotel' in obj['name']:
        print(index)

19


In [25]:
# 檢查 json 檔是否有 hotel_id 相同的資料
id_to_indices = {}  # 建立空字典
for i, d in enumerate(listResult):
    hotel_id = d['hotel_id']
    if hotel_id not in id_to_indices:
        id_to_indices[hotel_id] = [i]
    else:
        id_to_indices[hotel_id].append(i)

duplicates = {hotel_id: indices for hotel_id, indices in id_to_indices.items() if len(indices) > 1}

print(duplicates)


{}
